In [ ]:
from mssm.models import *
from mssmViz.sim import *
from mssmViz.plot import *
from mssm.src.python.utils import DummyRhoPrior
import pickle
import copy
import os
from src.utils import GAMLSSGENSMOOTHFamily,llk_gamm_fun,init_lambda,correct_VB

try:
    os.makedirs("./results")
except:
    print("Result directories already exist.")

try:
    os.makedirs("./results/sim")
    os.makedirs("./results/data")
    os.makedirs("./results/plots")
    os.makedirs("./results/data/sim1")
    os.makedirs("./results/data/sim2")
    os.makedirs("./results/data/sim3")
    os.makedirs("./results/data/sim1/plots")
    os.makedirs("./results/data/sim3/plots")
    os.makedirs("./results/data/sim4")
    os.makedirs("./results/data/sim5")
except:
    print("Result sub-directory already exist.")

n_cores = 20
prior = None#DummyRhoPrior(b=np.log(1e7))

# Overwrite correct_VB with lower bounds used for paper
import mssm
mssm.src.python.compare.correct_VB = correct_VB

# Now load compare
from mssm.src.python.compare import compare_CDL

In [ ]:
############################# Simulation 5 #############################
n_c = 10
n_sim = 100
df = 40
nR = 250

families = ["Gaussian","Binomial","Gamma"]
binom_offsets = [0,-5,0]
scales = [2,2,2]

for family,binom_offset,scale in zip(families,binom_offsets,scales):

    Failures = np.zeros(n_c)
    AIC_rej = np.zeros(n_c)
    AIC_t1_rej = np.zeros(n_c)

    eta_mses = np.zeros((n_c,n_sim,4))

    # WPS-like
    WPS_AIC_PQL_rej = np.zeros(n_c)

    # qEFS
    AIC_qEFS_rej = np.zeros(n_c)
    WPS_AIC_PQL_qEFS_rej = np.zeros(n_c)

    # Expected AICs
    edf_PQL_REML_JJJ3_AIC_rej  = np.zeros(n_c)
    edf_PQL_NORM_JJJ2_AIC_rej = np.zeros(n_c)


    for c_i,c_val in enumerate(np.linspace(0,1,n_c)):

        iterator = tqdm(range(n_sim),desc="Simulating",leave=True)
        for sim_i in iterator:

            ######################################## Family setup ########################################
            mod_fam = Gaussian()
            mod_fam2 =GAUMLSS([Identity(),LOGb(-0.01)])
            if family == "Binomial":
                mod_fam = Binomial()
                mod_fam2 = MULNOMLSS(1)
            if family == "Gamma":
                mod_fam = Gamma()
                mod_fam2 = GAMMALS([LOG(),LOGb(-0.01)])

            mod_fam.is_canonical = True

            ######################################## GSMM set up ########################################
            gsmm_fam = GAMLSSGENSMOOTHFamily(2 if family != "Binomial" else 1,copy.deepcopy(mod_fam2.links),llk_gamm_fun,copy.deepcopy(mod_fam2))
            
            if family != "Binomial":
                gsmm_fam.init_coef = lambda models: np.array([1e-4 for _ in range(models[0].formula.n_coef+1)]).reshape(-1,1)
            else:
                gsmm_fam.init_coef = lambda models: np.array([1e-4 for _ in range(models[0].formula.n_coef)]).reshape(-1,1)
            
            gsmm2_fam = copy.deepcopy(gsmm_fam)
            gsmm_fam.init_lambda = init_lambda
            gsmm2_fam.init_lambda = init_lambda
            

            ######################################## Create Data ########################################
            sim_fit_dat = sim4(n=500,scale=scale,c=c_val,family=mod_fam,binom_offset=binom_offset,seed=sim_i)
            sim_fit_dat.to_csv(f'./results/data/sim5/sim_size:{n_sim}_c:{c_i}_fam:{family}_nR:{nR}_df:{df}_scale:{scale}_binomoffset:{binom_offset}_set:{sim_i}.csv',index=False)

            ######################################## First Formula + Models ########################################
            sim_fit_formula = Formula(lhs("y"),
                                        [i(),f(["x0"],nk=9),f(["x1"],nk=9),f(["x2"],nk=9),f(["x3"],nk=9),ri("x4")],
                                        data=sim_fit_dat,
                                        print_warn=False)
            
            sim_fit_formula_sd = Formula(lhs("y"),
                                        [i()],data=sim_fit_dat,
                                            print_warn=False)
            
            if family != "Binomial":
                gsmm_formulas = copy.deepcopy([sim_fit_formula,sim_fit_formula_sd])
            else:
                gsmm_formulas = copy.deepcopy([sim_fit_formula])
            
            gsmm_model = GSMM(formulas=gsmm_formulas,family=gsmm_fam)
            sim_fit_model = GAMM(sim_fit_formula,mod_fam)

            sim_fit_model.fit(exclude_lambda=False,extend_lambda=False,progress_bar=False,max_outer=1000,max_inner=1)
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                bfgs_opt={"gtol":1e-9,
                        "ftol":1e-9,
                        "maxcor":30,
                        "maxls":200,
                        "maxfun":1e7}
                
                gsmm_model.fit(init_coef=None,method='qEFS',extend_lambda=False,
                                   control_lambda=False,max_outer=200,max_inner=500,min_inner=500,
                                   seed=sim_i,qEFSH='SR1',max_restarts=0,overwrite_coef=False,qEFS_init_converge=False,prefit_grad=True,
                                   progress_bar=False,repara=True,**bfgs_opt)

                # Get linear predictors for gsmm_model
                if family != "Binomial":
                    split_coef = np.split(gsmm_model.overall_coef,gsmm_model.coef_split_idx)
                else:
                    split_coef = [gsmm_model.overall_coef]

                Xs = gsmm_model.get_mmat()
                gsmm_model.pred = Xs[0]@split_coef[0].reshape(-1,1)
            
            ######################################## Second Formula + Models ########################################
            sim_fit_formula2 = Formula(lhs("y"),
                                        [i(),f(["x0"],nk=9),f(["x1"],nk=9),f(["x2"],nk=9),f(["x3"],nk=9)],
                                        data=sim_fit_dat,
                                        print_warn=False)
            
            if family != "Binomial":
                gsmm2_formulas = copy.deepcopy([sim_fit_formula2,sim_fit_formula_sd])
            else:
                gsmm2_formulas = copy.deepcopy([sim_fit_formula2])
            
            gsmm2_model = GSMM(formulas=gsmm2_formulas,family=gsmm2_fam)
            sim_fit_model2 = GAMM(sim_fit_formula2,mod_fam)
            
            sim_fit_model2.fit(exclude_lambda=False,extend_lambda=False,progress_bar=False,max_outer=1000,max_inner=1)
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                bfgs_opt={"gtol":1e-9,
                        "ftol":1e-9,
                        "maxcor":30,
                        "maxls":200,
                        "maxfun":1e7}
                
                gsmm2_model.fit(init_coef=None,method='qEFS',extend_lambda=False,
                                   control_lambda=False,max_outer=200,max_inner=500,min_inner=500,
                                   seed=sim_i,qEFSH='SR1',max_restarts=0,overwrite_coef=False,qEFS_init_converge=False,prefit_grad=True,
                                   progress_bar=False,repara=True,**bfgs_opt)

                # Get linear predictors for gsmm_model
                if family != "Binomial":
                    split_coef = np.split(gsmm2_model.overall_coef,gsmm2_model.coef_split_idx)
                else:
                    split_coef = [gsmm2_model.overall_coef]

                Xs = gsmm2_model.get_mmat()
                gsmm2_model.pred = Xs[0]@split_coef[0].reshape(-1,1)
            
            
            try:
                for mi,model in enumerate([sim_fit_model,sim_fit_model2,gsmm_model,gsmm2_model]): #
                    pred_diff = model.pred.flatten() - sim_fit_dat["eta"].values
                    eta_mses[c_i,sim_i,mi] = np.dot(pred_diff,pred_diff)/len(pred_diff)
                    
            except:
                print("Error collecting MSEs")

            ######################################## Comparisons ########################################
            try:
                uncor_result = compare_CDL(sim_fit_model,sim_fit_model2,correct_V=False,correct_t1=True,grid='JJJ3',seed=sim_i,verbose=False,only_expected_edf=False)
                
                # Compute heuristic upper edf from WPS (2016) as well
                llk1 = uncor_result["aic1"] - 2*uncor_result["DOF12"]
                llk2 = uncor_result["aic2"] - 2*uncor_result["DOF22"] 
                aic1_t1 =  llk1 + 2*uncor_result["DOF1"]
                aic2_t1 =  llk2 + 2*uncor_result["DOF2"]

                aic_diff_t1 = aic1_t1 - aic2_t1

                # WPS:
                wps_pql_result = compare_CDL(sim_fit_model,sim_fit_model2,nR=nR,df=df,correct_t1=False,n_c=n_cores,grid='JJJ1',Vp_fidiff=False,seed=sim_i,verbose=False,only_expected_edf=False)
                
                # Expected
                if family == "Gaussian":
                    pql_NORM_JJJ2_edf_result = compare_CDL(sim_fit_model,sim_fit_model2,nR=nR,df=np.inf,correct_t1=False,n_c=n_cores,grid='JJJ2',Vp_fidiff=False,only_expected_edf=False,seed=sim_i,verbose=False,use_importance_weights=False)
                else:
                    pql_REML_JJJ3_edf_result = compare_CDL(sim_fit_model,sim_fit_model2,nR=nR,df=df,correct_t1=False,n_c=n_cores,grid='JJJ3',Vp_fidiff=False,only_expected_edf=False,seed=sim_i,verbose=False,use_importance_weights=True,recompute_H=True,prior=prior)
                
                # GSMM
                uncor_qEFS_result = compare_CDL(gsmm_model,gsmm2_model,correct_V=False,correct_t1=False,grid='JJJ3',seed=sim_i,verbose=False,only_expected_edf=False)
                wps_pql_qEFS_result = compare_CDL(gsmm_model,gsmm2_model,nR=nR,df=df,correct_t1=False,n_c=n_cores,grid='JJJ1',Vp_fidiff=False,seed=sim_i,verbose=False,only_expected_edf=False)

                if uncor_result["aic_diff"] < 0:
                    AIC_rej[c_i] += 1
                if aic_diff_t1 < 0:
                    AIC_t1_rej[c_i] += 1
                
                # WPS
                if wps_pql_result["aic_diff"] < 0:
                    WPS_AIC_PQL_rej[c_i] += 1

                # Expected:
                if family == "Gaussian" and pql_NORM_JJJ2_edf_result["aic_diff"] < 0:
                    edf_PQL_NORM_JJJ2_AIC_rej[c_i] += 1
                if family != "Gaussian" and pql_REML_JJJ3_edf_result["aic_diff"] < 0:
                    edf_PQL_REML_JJJ3_AIC_rej[c_i] += 1
                
                # QEFS:
                if uncor_qEFS_result["aic_diff"] < 0:
                    AIC_qEFS_rej[c_i] += 1
                if wps_pql_qEFS_result["aic_diff"] < 0:
                    WPS_AIC_PQL_qEFS_rej[c_i] += 1
                

            except:
                print(f"Failure for family: {family} at sim: {[sim_i,c_i]} in compare code")
                Failures[c_i] += 1

            # Show some progress info
            progress = [np.round(AIC_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                        np.round(WPS_AIC_PQL_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                        np.round(AIC_qEFS_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                        np.round(WPS_AIC_PQL_qEFS_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                        np.round(edf_PQL_REML_JJJ3_AIC_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                        np.round(edf_PQL_NORM_JJJ2_AIC_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2)]
            
            iterator.set_description_str(desc=f"Acc (UC, WPS, qEFS, qEFS_WPS, UpperR, UpperN).: {progress}", refresh=True)

        iterator.close()
    
    ######################################## Save & Plot ########################################
    print(Failures)
    completed_sim = (n_sim - Failures)

    AIC_rej /= completed_sim
    AIC_t1_rej /= completed_sim
    WPS_AIC_PQL_rej /= completed_sim
    WPS_AIC_PQL_qEFS_rej /= completed_sim
    AIC_qEFS_rej /= completed_sim
    edf_PQL_REML_JJJ3_AIC_rej /= completed_sim
    edf_PQL_NORM_JJJ2_AIC_rej /= completed_sim
    
    sim_outcome = {"AIC_rej":AIC_rej,
                    "AIC_t1_rej":AIC_t1_rej,
                    "WPS_AIC_PQL_rej":WPS_AIC_PQL_rej,
                    "AIC_qEFS_rej":AIC_qEFS_rej,
                    "WPS_AIC_PQL_qEFS_rej":WPS_AIC_PQL_qEFS_rej,
                    "edf_PQL_REML_JJJ3_AIC_rej":edf_PQL_REML_JJJ3_AIC_rej,
                    "edf_PQL_NORM_JJJ2_AIC_rej":edf_PQL_NORM_JJJ2_AIC_rej,
                    "Failures":Failures,
                    "MSE":eta_mses}

    with open(f'./results/sim/sim_5_size:{n_sim}_n_c:{n_c}_fam:{family}_nR:{nR}_df:{df}_scale:{scale}_binomoffset:{binom_offset}.pickle', 'wb') as file:
        pickle.dump(sim_outcome,file, protocol=pickle.HIGHEST_PROTOCOL)

    plt.plot(np.linspace(0,1,n_c),AIC_rej,label='AIC')
    plt.plot(np.linspace(0,1,n_c),AIC_t1_rej,label='AIC t1')
    plt.plot(np.linspace(0,1,n_c),WPS_AIC_PQL_rej,label='AIC WPS PQL')
    plt.plot(np.linspace(0,1,n_c),AIC_qEFS_rej,label='AIC GSMM')
    plt.plot(np.linspace(0,1,n_c),WPS_AIC_PQL_qEFS_rej,label='AIC WPS GSMM PQL')
    plt.plot(np.linspace(0,1,n_c),edf_PQL_REML_JJJ3_AIC_rej,label='AIC REML JJJ3')
    plt.plot(np.linspace(0,1,n_c),edf_PQL_NORM_JJJ2_AIC_rej,label='AIC NORM JJJ3')
    plt.legend()
    plt.show()


In [ ]:
############################# Simulation 5 #############################
n_c = 10
n_sim = 100
df = 40
nR = 250
scale = 2

bfgs_opt={"gtol":1e-9,
          "ftol":1e-9,
          "maxcor":30,
          "maxls":200,
          "maxfun":1e7}

Failures = np.zeros(n_c)
AIC_rej = np.zeros(n_c)
AIC_t1_rej = np.zeros(n_c)

eta_mses = np.zeros((n_c,n_sim,4))

# WPS-like
WPS_AIC_PQL_rej = np.zeros(n_c)

# qEFS
AIC_qEFS_rej = np.zeros(n_c)
WPS_AIC_PQL_qEFS_rej = np.zeros(n_c)

# Expected AICs
edf_PQL_REML_JJJ3_AIC_rej  = np.zeros(n_c)

for c_i,c_val in enumerate(np.linspace(0,1,n_c)):


    iterator = tqdm(range(n_sim),desc="Simulating",leave=True)
    for sim_i in iterator:

        ######################################## Create Data ########################################
        sim_fit_dat = sim4(n=500,scale=scale,c=c_val,family=PropHaz([0],[0]),binom_offset=0.1,seed=sim_i)
        sim_fit_dat = sim_fit_dat.sort_values(['y'],ascending=[False])
        sim_fit_dat = sim_fit_dat.reset_index(drop=True)

        u,inv = np.unique(sim_fit_dat["y"],return_inverse=True)
        ut = np.flip(u)
        r = np.abs(inv - max(inv))
        
        sim_fit_dat.to_csv(f'./results/data/sim5/sim_size:{n_sim}_c:{c_i}_fam:{"PropHaz"}_nR:{nR}_df:{df}_scale:{scale}_set:{sim_i}.csv',index=False)

        ######################################## First Formula + Models ########################################
        sim_fit_formula = Formula(lhs("delta"),
                                    [f(["x0"],nk=9),f(["x1"],nk=9),f(["x2"],nk=9),f(["x3"],nk=9),ri("x4")],
                                    data=sim_fit_dat,
                                    print_warn=False)
        
        sim_formulas = [sim_fit_formula]

        gsmm_newton_fam = PropHaz(ut,r)
        gsmm_newton_fam.init_lambda = init_lambda
        sim_fit_model = GSMM(copy.deepcopy(sim_formulas),gsmm_newton_fam)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sim_fit_model.fit(init_coef=None,method="QR/Chol",extend_lambda=False,
                          control_lambda=False,max_outer=200,seed=sim_i,max_inner=500,
                          min_inner=500,prefit_grad=False,repara=True,progress_bar=False)
            
            # Get linear predictors for gsmm_model
            split_coef = [sim_fit_model.overall_coef]

            Xs = sim_fit_model.get_mmat()
            sim_fit_model.pred = Xs[0]@split_coef[0].reshape(-1,1)
        
        
        gsmm_qefs_fam = PropHaz(ut,r)
        gsmm_qefs_fam.init_lambda = init_lambda
        gsmm_model = GSMM(copy.deepcopy(sim_formulas),gsmm_qefs_fam)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            bfgs_opt={"gtol":1e-9,
                        "ftol":1e-9,
                        "maxcor":30,
                        "maxls":200,
                        "maxfun":1e7}

            gsmm_model.fit(init_coef=None,method='qEFS',extend_lambda=False,
                                   control_lambda=False,max_outer=200,max_inner=500,min_inner=500,
                                   seed=sim_i,qEFSH='SR1',max_restarts=0,overwrite_coef=False,qEFS_init_converge=False,prefit_grad=True,
                                   progress_bar=False,repara=True,**bfgs_opt)
            
            # Get linear predictors for gsmm_model
            split_coef = [gsmm_model.overall_coef]

            Xs = gsmm_model.get_mmat()
            gsmm_model.pred = Xs[0]@split_coef[0].reshape(-1,1)
        
        
        ######################################## Second Formula + Models ########################################
        sim_fit_formula2 = Formula(lhs("delta"),
                                    [f(["x0"],nk=9),f(["x1"],nk=9),f(["x2"],nk=9),f(["x3"],nk=9)],
                                    data=sim_fit_dat,
                                    print_warn=False)
        
        sim_formulas2 = [sim_fit_formula2]

        gsmm_newton_fam2 = PropHaz(ut,r)
        gsmm_newton_fam2.init_lambda = init_lambda
        sim_fit_model2 = GSMM(copy.deepcopy(sim_formulas2),gsmm_newton_fam2)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sim_fit_model2.fit(init_coef=None,method="QR/Chol",extend_lambda=False,
                          control_lambda=False,max_outer=200,seed=sim_i,max_inner=500,
                          min_inner=500,prefit_grad=False,repara=True,progress_bar=False)
            
            # Get linear predictors for gsmm_model
            split_coef = [sim_fit_model2.overall_coef]

            Xs = sim_fit_model2.get_mmat()
            sim_fit_model2.pred = Xs[0]@split_coef[0].reshape(-1,1)
        
        
        gsmm_qefs_fam2 = PropHaz(ut,r)
        gsmm_qefs_fam2.init_lambda = init_lambda
        gsmm2_model = GSMM(copy.deepcopy(sim_formulas2),gsmm_qefs_fam2)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            bfgs_opt={"gtol":1e-9,
                        "ftol":1e-9,
                        "maxcor":30,
                        "maxls":200,
                        "maxfun":1e7}

            gsmm2_model.fit(init_coef=None,method='qEFS',extend_lambda=False,
                                   control_lambda=False,max_outer=200,max_inner=500,min_inner=500,
                                   seed=sim_i,qEFSH='SR1',max_restarts=0,overwrite_coef=False,qEFS_init_converge=False,prefit_grad=True,
                                   progress_bar=False,repara=True,**bfgs_opt)
            
            # Get linear predictors for gsmm_model
            split_coef = [gsmm2_model.overall_coef]

            Xs = gsmm2_model.get_mmat()
            gsmm2_model.pred = Xs[0]@split_coef[0].reshape(-1,1)
        
        try:
            for mi,model in enumerate([sim_fit_model,sim_fit_model2,gsmm_model,gsmm2_model]): #
                pred_diff = model.pred.flatten() - sim_fit_dat["eta"].values
                eta_mses[c_i,sim_i,mi] = np.dot(pred_diff,pred_diff)/len(pred_diff)
                
        except:
            print("Error collecting MSEs")
        
        ######################################## Comparisons ########################################
        try:
            uncor_result = compare_CDL(sim_fit_model,sim_fit_model2,correct_V=False,correct_t1=True,grid='JJJ3',seed=sim_i,verbose=False,only_expected_edf=False)
            
            # Compute heuristic upper edf from WPS (2016) as well
            llk1 = uncor_result["aic1"] - 2*uncor_result["DOF12"]
            llk2 = uncor_result["aic2"] - 2*uncor_result["DOF22"] 
            aic1_t1 =  llk1 + 2*uncor_result["DOF1"]
            aic2_t1 =  llk2 + 2*uncor_result["DOF2"]

            aic_diff_t1 = aic1_t1 - aic2_t1

            # WPS:
            wps_pql_result = compare_CDL(sim_fit_model,sim_fit_model2,nR=nR,df=df,correct_t1=False,n_c=n_cores,grid='JJJ1',Vp_fidiff=False,seed=sim_i,verbose=False,only_expected_edf=False)
            
            # Expected
            pql_REML_JJJ3_edf_result = compare_CDL(sim_fit_model,sim_fit_model2,nR=nR,df=df,correct_t1=False,n_c=n_cores,grid='JJJ3',Vp_fidiff=False,only_expected_edf=False,seed=sim_i,verbose=False,use_importance_weights=True,recompute_H=True,prior=prior)
            
            # GSMM
            uncor_qEFS_result = compare_CDL(gsmm_model,gsmm2_model,correct_V=False,correct_t1=False,grid='JJJ3',seed=sim_i,verbose=False,only_expected_edf=False)
            wps_pql_qEFS_result = compare_CDL(gsmm_model,gsmm2_model,nR=nR,df=df,correct_t1=False,n_c=n_cores,grid='JJJ1',Vp_fidiff=False,seed=sim_i,verbose=False,only_expected_edf=False)

            if uncor_result["aic_diff"] < 0:
                AIC_rej[c_i] += 1
            if aic_diff_t1 < 0:
                AIC_t1_rej[c_i] += 1
            
            # WPS
            if wps_pql_result["aic_diff"] < 0:
                WPS_AIC_PQL_rej[c_i] += 1

            # Expected:
            if pql_REML_JJJ3_edf_result["aic_diff"] < 0:
                edf_PQL_REML_JJJ3_AIC_rej[c_i] += 1
            
            # QEFS:
            if uncor_qEFS_result["aic_diff"] < 0:
                AIC_qEFS_rej[c_i] += 1
            if wps_pql_qEFS_result["aic_diff"] < 0:
                WPS_AIC_PQL_qEFS_rej[c_i] += 1
            
            
        except:
            print(f"Failure for family: {'PropHaz'} at sim: {[sim_i,c_i]} in compare code")
            Failures[c_i] += 1

        # Show some progress info
        progress = [np.round(AIC_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                    np.round(WPS_AIC_PQL_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                    np.round(AIC_qEFS_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                    np.round(WPS_AIC_PQL_qEFS_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2),
                    np.round(edf_PQL_REML_JJJ3_AIC_rej[c_i]/(sim_i + 1 - Failures[c_i]),decimals=2)]
        
        iterator.set_description_str(desc=f"Acc (UC, WPS, qEFS, qEFS_WPS, UpperR).: {progress}", refresh=True)

    iterator.close()

######################################## Save & Plot ########################################
print(Failures)
completed_sim = (n_sim - Failures)

AIC_rej /= completed_sim
AIC_t1_rej /= completed_sim
WPS_AIC_PQL_rej /= completed_sim
WPS_AIC_PQL_qEFS_rej /= completed_sim
AIC_qEFS_rej /= completed_sim
edf_PQL_REML_JJJ3_AIC_rej /= completed_sim

sim_outcome = {"AIC_rej":AIC_rej,
                "AIC_t1_rej":AIC_t1_rej,
                "WPS_AIC_PQL_rej":WPS_AIC_PQL_rej,
                "AIC_qEFS_rej":AIC_qEFS_rej,
                "WPS_AIC_PQL_qEFS_rej":WPS_AIC_PQL_qEFS_rej,
                "edf_PQL_REML_JJJ3_AIC_rej":edf_PQL_REML_JJJ3_AIC_rej,
                "Failures":Failures,
                "MSE":eta_mses}

with open(f'./results/sim/sim_5_size:{n_sim}_n_c:{n_c}_fam:{"PropHaz"}_nR:{nR}_df:{df}_scale:{scale}.pickle', 'wb') as file:
    pickle.dump(sim_outcome,file, protocol=pickle.HIGHEST_PROTOCOL)

plt.plot(np.linspace(0,1,n_c),AIC_rej,label='AIC')
plt.plot(np.linspace(0,1,n_c),AIC_t1_rej,label='AIC t1')
plt.plot(np.linspace(0,1,n_c),WPS_AIC_PQL_rej,label='AIC WPS PQL')
plt.plot(np.linspace(0,1,n_c),AIC_qEFS_rej,label='AIC GSMM')
plt.plot(np.linspace(0,1,n_c),WPS_AIC_PQL_qEFS_rej,label='AIC WPS GSMM PQL')
plt.plot(np.linspace(0,1,n_c),edf_PQL_REML_JJJ3_AIC_rej,label='AIC REML JJJ3')
plt.legend()
plt.show()
